In [1]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from PIL import Image
import numpy as np
import json
import cv2
import time
from sklearn.metrics import confusion_matrix, precision_score, recall_score, log_loss
from concurrent.futures import ThreadPoolExecutor

from utils import print_progress_bar

In [3]:
all_labels = ['nature', 'country','city']
model_type = "ImageClassifier" # Choose one amongst ("SimpleMLP", "Transformer", "ImageClassifier").
path = "../datasets/classified_pictures/entropy_results_augmented.json"
test_part = 0.05
epochs = 2
batch_size = 64
parallel_jobs = 5
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda:0


In [3]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, queries):
        N = queries.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], queries.shape[1]

        # Split the embedding into self.head different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = queries.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        # Scaled dot-product attention
        scores = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        scores = scores / (self.head_dim ** 0.5)
        attention = torch.nn.functional.softmax(scores, dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out


# Define the Transformer Block
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, 2 * embed_size),
            nn.ReLU(),
            nn.Linear(2 * embed_size, embed_size),
        )
        self.dropout = nn.Dropout(0.1)
        self.batch_norm1 = nn.BatchNorm1d(embed_size)
        self.batch_norm2 = nn.BatchNorm1d(embed_size)

    def forward(self, value, key, query):
        attention = self.attention(value, key, query)

        # Reshape tensor before BatchNorm1d
        reshaped_attention = attention.view(-1, attention.size(-1))

        x = self.batch_norm1(self.norm1(reshaped_attention))
        x = x.view(attention.size())  # Reshape back to original size
        x = self.dropout(x)

        forward = self.feed_forward(x)

        # Reshape tensor before second BatchNorm1d
        reshaped_forward = forward.view(-1, forward.size(-1))

        out = self.batch_norm2(self.norm2(reshaped_forward))
        out = out.view(forward.size())  # Reshape back to original size

        return out


class Classifier(nn.Module):
    def __init__(self, num_classes):
        super(Classifier, self).__init__()
        self.encoder1 = nn.Embedding(256, 512)
        self.encoder2 = nn.Embedding(256, 512)
        self.transformer1 = TransformerBlock(embed_size=512, heads=8)
        self.transformer2 = TransformerBlock(embed_size=512, heads=8)
        self.classifier = nn.Linear(1024, num_classes)
        self.possible_labels = all_labels

    def forward(self, x1, x2):
        x1 = self.encoder1(x1)
        x2 = self.encoder2(x2)
        x1 = self.transformer1(x1, x1, x1)
        x2 = self.transformer2(x2, x2, x2)
        # Padding x1 to match the sequence length of x2
        if x1.size(1) < x2.size(1):
            padding = torch.zeros(x1.size(0), x2.size(1) - x1.size(1), x1.size(2)).to(x1.device)
            x1 = torch.cat((x1, padding), dim=1)
        # Padding x2 to match the sequence length of x1
        elif x2.size(1) < x1.size(1):
            padding = torch.zeros(x2.size(0), x1.size(1) - x2.size(1), x2.size(2)).to(x2.device)
            x2 = torch.cat((x2, padding), dim=1)

        x = torch.cat((x1, x2), dim=2)
        x = self.classifier(x[:, 0, :])
        return x

    def train_model(self, dataset, epochs=100):
        loss = None
        entropies = [torch.tensor(d['entropies'], dtype=torch.long) for d in dataset]
        dwt_entropies = [torch.tensor(d['dwt'], dtype=torch.long) for d in dataset]
        labels = [self.possible_labels.index(d['label']) for d in dataset]
        entropies = torch.stack(entropies)
        dwt_entropies = torch.stack(dwt_entropies)
        labels = torch.tensor(labels, dtype=torch.long)
        train_data = TensorDataset(entropies, dwt_entropies, labels)
        train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=0.0001)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

        for epoch in range(epochs):
            for batch_idx, (data1, data2, target) in enumerate(train_loader):
                self.train()
                optimizer.zero_grad()
                output = self(data1, data2)
                loss = criterion(output, target)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=1)
                optimizer.step()
            scheduler.step(loss.item())
            print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

    def predict(self, numbers1, numbers2):
        self.eval()
        with torch.no_grad():
            numbers1 = torch.tensor(numbers1, dtype=torch.long).unsqueeze(0)
            numbers2 = torch.tensor(numbers2, dtype=torch.long).unsqueeze(0)
            output_ = self(numbers1, numbers2)
            predicted_label_idx = torch.argmax(output_, dim=1).item()
            return self.possible_labels[predicted_label_idx]

In [4]:
class MultiLabelImageClassifier(nn.Module):
    def __init__(self, num_classes, device=None):
        super(MultiLabelImageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3)
        self.bn3 = nn.BatchNorm2d(128)
        self.fc1 = nn.Linear(128 * 6 * 6, 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.possible_labels = ['nature', 'country','city']
        self.device = device

    def forward(self, x):
        x = nn.functional.relu(self.bn1(self.conv1(x)))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.bn2(self.conv2(x)))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.bn3(self.conv3(x)))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.adaptive_avg_pool2d(x, (6, 6))
        x = x.view(-1, 128 * 6 * 6)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x 
    
    def train_model(self, dataset, epochs=100, batch_size=64, lr=0.01):
        self.to(self.device)  # Move the model to the device
        
        t = time.time()
        train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(self.parameters(), lr=lr)
        n = len(train_loader)

        for epoch in range(epochs):
            print_progress_bar('Batches processed', 1, n, start_time=t)
            for batch_idx, batch in enumerate(train_loader):
                data = batch['image'].float().to(self.device) 
                target = batch['label']
                target = torch.tensor([one_hot_encode(t, self.possible_labels) for t in target], dtype=torch.float32).to(self.device)  # Move target to the device

                
                # Debugging: Print shapes
                # print(f"\nData shape: {data.shape}, Target shape: {target.shape}")

                self.train()
                optimizer.zero_grad()
                output = self(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
                
                print_progress_bar('Batches processed', batch_idx+1, n, start_time=t)
            

            print(f"\n Epoch {epoch + 1}, Loss: {loss.item()}")

    def predict(self, image):
        image = image.to(self.device)
        with torch.no_grad():
            image = image.unsqueeze(0).float()
            output_ = self(image)
            probabilities = torch.softmax(output_, dim=1)
            max_index = torch.argmax(probabilities)
            label_prob_dict = {label: prob.item() for label, prob in zip(self.possible_labels, probabilities[0])}
            
        return str(self.possible_labels[max_index]), label_prob_dict


In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, dwt_input_dim, num_classes):
        super(SimpleMLP, self).__init__()
        self.layer1 = nn.Linear(input_dim, 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, num_classes)
        self.dwt_layer = nn.Linear(dwt_input_dim, 128) if dwt_input_dim else None
        self.dropout = nn.Dropout(0.5)
        self.possible_labels = all_labels

    def forward(self, x, dwt_x=None):
        x = F.relu(self.layer1(x))
        x = self.dropout(x)
        if dwt_x is not None and self.dwt_layer:
            dwt_x = F.relu(self.dwt_layer(dwt_x))
            x += dwt_x
        x = F.relu(self.layer2(x))
        x = self.dropout(x)
        x = F.softmax(self.layer3(x), dim=1)
        return x

    def train_model(self, dataset, epochs=100):
        loss = None
        entropies = [torch.tensor(d['entropies'], dtype=torch.float) for d in dataset]
        labels = [torch.tensor([1 if l in d['label'] else 0 for l in self.possible_labels], dtype=torch.float) for d in
                  dataset]
        entropies = torch.stack(entropies)
        labels = torch.stack(labels)
        train_data = TensorDataset(entropies, labels)
        train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
        criterion = nn.BCELoss()
        optimizer = optim.Adam(self.parameters(), lr=0.001)

        for epoch in range(epochs):
            for batch_idx, (data, target) in enumerate(train_loader):
                self.train()
                optimizer.zero_grad()
                output = self(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
            print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

    def predict(self, entropies, dwt=None):
        self.eval()
        with torch.no_grad():
            entropies = torch.tensor(entropies, dtype=torch.float).unsqueeze(0)
            if dwt is not None and self.dwt_layer:
                dwt = torch.tensor(dwt, dtype=torch.float).unsqueeze(0)
            else:
                dwt = None
            output_ = self(entropies, dwt)
            predicted_label_idx = torch.argmax(output_, dim=1).item()
            return self.possible_labels[predicted_label_idx]

In [1]:
def load_image(entry):
    image_path = entry['path']
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not read image at {image_path}")
        return None
    resized_image = cv2.resize(image, (224, 224))
    resized_image = torch.tensor(resized_image, dtype=torch.float32).permute(2, 0, 1) / 255.0
    label = entry['label']
    if isinstance(label, str):
        label = [label]
    return {'image': resized_image, 'label': label}

def process_json(path, test_part, model_type="SimpleMLP", device='cpu'):
    with open(path, 'r') as f:
        metadata = json.load(f)
    dataset = []

    t = time.time()
    i = 0
    n = len(metadata)

    if model_type == "ImageClassifier":
        with ThreadPoolExecutor(max_workers=parallel_jobs) as executor:
            results = list(executor.map(load_image, metadata))
        dataset = [result for result in results if result is not None]
    else:
        for entry in metadata:
            i += 1
            entropies = [s['result'] for s in entry['entropy_results'] if s['method'] != 'dwt']
            dwt_entropies = next((s['result'][:9] for s in entry['entropy_results'] if s['method'] == 'dwt'), None)
            dataset.append({'entropies': entropies, 'dwt': dwt_entropies, 'label': entry['label']})
            print_progress_bar('Processed entry', i, n, t)

    print(f"\nLength of dataset: {len(dataset)}")

    if isinstance(test_part, float):
        i = int(test_part * len(dataset))
    elif isinstance(test_part, str):
        i = int(test_part)
    else:
        raise ValueError("Incompatible format for 'test_part'.")

    test_set = dataset[-i:]
    dataset = dataset[:-i]

    if model_type != "ImageClassifier":
        input_dim = len(dataset[0]['entropies'])
        dwt_input_dim = len(dataset[0]['dwt'])
    else:
        input_dim = None
        dwt_input_dim = None

    # Debugging print statements
    print(f"Total number of entries in the dataset: {len(dataset)}")
    print(f"Total number of entries in the test set: {len(test_set)}")    
    if model_type != "ImageClassifier":
        print(f"Input dimension for entropies: {input_dim}")
        print(f"Input dimension for dwt: {dwt_input_dim}")
    else:
        print("Model type is ImageClassifier, skipping input dimensions.")

    return dataset, test_set, input_dim, dwt_input_dim

In [12]:
def evaluate_model(model, test_set, model_type="SimpleMLP"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Define the device based on availability
    model.to(device)  # Move the model to the device
    
    stats = {'test_samples': 0, 'right_predictions': 0}
    y_true = []
    y_pred = []
    y_prob = []
    
    for test in test_set:
        stats['test_samples'] += 1
        
        if model_type == "ImageClassifier":
            image = test['image'].to(device)  # Move the image to the device
            predicted_label, label_probs = model.predict(image)
        else:
            entropies = test['entropies'].to(device)  # Move entropies to the device
            dwt = test['dwt'].to(device)  # Move dwt to the device
            predicted_label, label_probs = model.predict(entropies, dwt)
        
        y_true.append(test["label"])
        y_pred.append(predicted_label)
        y_prob.append(label_probs[test["label"][0]])
        
        if predicted_label == test["label"]:
            stats['right_predictions'] += 1
            print(f'Predicted label: {predicted_label}.  Real label: {test["label"]}. Prediction correct!')
        else:
            print(f'Predicted label: {predicted_label}.  Real label: {test["label"]}. False prediction.')
    
    # Calculate additional metrics
    conf_matrix = confusion_matrix(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    logloss = log_loss(y_true, np.array(y_prob))
    
    stats['success_rate'] = 100 * stats['right_predictions'] / stats['test_samples']
    stats['confusion_matrix'] = conf_matrix
    stats['precision'] = precision
    stats['recall'] = recall
    stats['log_loss'] = logloss
    
    print(f"{stats['right_predictions']} samples out of {stats['test_samples']} were predicted correctly.\n"
          f"The model's success rate is: {stats['success_rate']}%")
    print(f"Confusion Matrix: \n{conf_matrix}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"Log Loss: {logloss}")

In [13]:
def one_hot_encode(labels, all_labels_):
    if not isinstance(labels, list):
        labels = [labels]
    num_labels = len(labels)
    return [1/num_labels if label in labels else 0 for label in all_labels_]


def custom_collate(batch):
    images = torch.stack([item['image'] for item in batch])
    labels = [item['label'] for item in batch]
    return {'image': images, 'label': labels}

In [15]:
dataset, test_set, input_dim, dwt_input_dim = process_json(path, test_part, model_type)
dataset_length = len(dataset)
num_classes = len(all_labels)
print(f"Total number of entries in the dataset: {dataset_length}")
print(f"Total number of entries in the test set: {len(test_set)}")    
print(f"Number of classes: {num_classes}")
print('Dataset processed.')
print(f"Dataset length: {dataset_length}")

Could not read image at ../datasets/classified_pictures/nature/map_image_23.9270_ 110.5920_20230827202537_rot90_flip1.png
Could not read image at ../datasets/classified_pictures/nature/map_image_23.9270_ 110.5920_20230827202537_rot180_flip0.png
Could not read image at ../datasets/classified_pictures/nature/map_image_23.9270_ 110.5920_20230827202537_rot270_flip1.png
Could not read image at ../datasets/classified_pictures/nature/map_image_23.9270_ 110.5920_20230827202537_rot180_flip1.png
Could not read image at ../datasets/classified_pictures/nature/map_image_39.4058_ -107.7400_20230821155051_rot0_flip1.png
Could not read image at ../datasets/classified_pictures/nature/map_image_39.4058_ -107.7400_20230821155051_rot90_flip1.png
Could not read image at ../datasets/classified_pictures/nature/map_image_39.4058_ -107.7400_20230821155051_rot270_flip1.png
Could not read image at ../datasets/classified_pictures/country/map_image_27.9465_ 114.1769_20230821155518_rot0_flip1.png
Could not read ima

KeyboardInterrupt: 

In [79]:
if model_type == "SimpleMLP":
    model = SimpleMLP(input_dim, dwt_input_dim, num_classes)   
elif model_type == "Transformer":
    model = Classifier(num_classes)    
elif model_type == "ImageClassifier":
    model = MultiLabelImageClassifier(num_classes, device=device)    
else:
    print("Invalid model type")    

In [80]:
model.train_model(dataset, epochs=epochs, batch_size=batch_size)
print('Model trained.')

Batches processed: ████---------------------------------------------- | 9.8% Complete 45/461 instances | Time: 01:37 | Time left: ~15:02

KeyboardInterrupt: 

In [ ]:
file_name = f"{model_type}_e={epochs}_ds={dataset_length}.pth"

In [ ]:
torch.save(model.state_dict(), file_name)
print("Model saved")

In [57]:
model.load_state_dict(torch.load(file_name, map_location=device))
model.eval()
print("model loaded")

model loaded


In [61]:
evaluate_model(model, test_set, model_type="ImageClassifier")

KeyError: 'country'